# 데이터 로딩
---

In [2]:
import pandas as pd

train_df = pd.read_csv('train_mec_v5.csv')
train_df.head(3)

,n_id,date,title,content,ord,info_,clean_content,clean_title,mecab_,mecab_title,last_pos_content,first_pos_content,last_pos_title,first_pos_title
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0,[이데일리 MARKETPOINT] time 현재 코스닥 기관 money 순매도,[마감]코스닥 기관 money 순매도,[ 이데일리 MARKETPOINT ] time 현재 코스닥 기관 money 순매도,[ 마감 ] 코스닥 기관 money 순매도,NNG,SSO,NNG,SSO
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1,'실적기반' 저가에 매집해야 할 month 급등유망주 rank 전격공개,[마감]코스닥 기관 money 순매도,' 실적 기반 ' 저가 매집 해야 할 month 급등 유망주 rank 전격 공개,[ 마감 ] 코스닥 기관 money 순매도,NNG,SY,NNG,SSO
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1,"하이스탁론, 선취수수료 없는 월 perc 최저금리 상품 출시",[마감]코스닥 기관 money 순매도,"하이스탁론 , 선취 수수료 없 는 월 perc 최저 금리 상품 출시",[ 마감 ] 코스닥 기관 money 순매도,NNG,NNP,NNG,SSO


## 결측치 확인
---
- 결측치가 3으로 나오는데, content에서 결측치를 처리하여 mecab_를 만들었기 때문에 분석을 하는 데이터에는 문제가 없음.

In [6]:
train_df.isnull().sum().sum()

3

# 전처리

## 새로운 변수 추가 (new, ord_div,counts)
---

**new 변수 (접두사 접미사 단어 결합)**
- title의 첫번째 단어 형태소, 마지막 단어 형태소, content의 첫번째 단어 형태소, 마지막 단어 형태소를 변수에 추가
- 형태소 변수를 추가한 이유는 가짜 기사와 진짜 기사의 첫번째,마지막 단어 형태소가 차이가남.

In [7]:
# new 변수 생성
train_df['new'] =  'FC'+train_df['first_pos_content']+" "+'FT'+train_df['first_pos_title']+" "+train_df['mecab_'] +" LC"+train_df['last_pos_content']+" LT"+train_df['last_pos_title']

**counts 변수 추가**
- 중복되는 문장의 갯수 변수

In [8]:
check = train_df[['title','content','info_']]
counts = pd.pivot_table(check,index=['content'],values='info_',aggfunc='count').reset_index()
counts.rename(columns = {'info_' : 'counts'},inplace=True)
mine = pd.merge(train_df,counts,how='left',on='content')
train_df['counts'] = mine['counts']

**ord_div 변수 추가**
- 문장내 기사의 위치 변수

In [9]:
# ord_div 변수 생성 
train_df['ord_div'] = train_df['ord'].div(train_df.groupby('n_id')['ord'].transform(lambda x: x.max()))
train_df.head()

,n_id,date,title,content,ord,info_,clean_content,clean_title,mecab_,mecab_title,last_pos_content,first_pos_content,last_pos_title,first_pos_title,new,counts,ord_div
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0,[이데일리 MARKETPOINT] time 현재 코스닥 기관 money 순매도,[마감]코스닥 기관 money 순매도,[ 이데일리 MARKETPOINT ] time 현재 코스닥 기관 money 순매도,[ 마감 ] 코스닥 기관 money 순매도,NNG,SSO,NNG,SSO,FCSSO FTSSO [ 이데일리 MARKETPOINT ] time 현재 코스닥 기...,1.0,0.250000
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1,'실적기반' 저가에 매집해야 할 month 급등유망주 rank 전격공개,[마감]코스닥 기관 money 순매도,' 실적 기반 ' 저가 매집 해야 할 month 급등 유망주 rank 전격 공개,[ 마감 ] 코스닥 기관 money 순매도,NNG,SY,NNG,SSO,FCSY FTSSO ' 실적 기반 ' 저가 매집 해야 할 month 급등 유망주 r...,542.0,0.500000
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1,"하이스탁론, 선취수수료 없는 월 perc 최저금리 상품 출시",[마감]코스닥 기관 money 순매도,"하이스탁론 , 선취 수수료 없 는 월 perc 최저 금리 상품 출시",[ 마감 ] 코스닥 기관 money 순매도,NNG,NNP,NNG,SSO,"FCNNP FTSSO 하이스탁론 , 선취 수수료 없 는 월 perc 최저 금리 상품...",542.0,0.750000
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0,종합 경제정보 미디어 이데일리 무단전재 & 재배포 금지,[마감]코스닥 기관 money 순매도,종합 경제 정보 미디어 이데일리 무단 전재 & 재배포 금지,[ 마감 ] 코스닥 기관 money 순매도,NNG,NNG,NNG,SSO,FCNNG FTSSO 종합 경제 정보 미디어 이데일리 무단 전재 & 재배포 금지 L...,605.0,1.000000
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0,전국적인 소비 붐 조성에 기여할 예정,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국 적 인 소비 붐 조성 기여 할 예정,"롯데 · 공영 등 7 개 TV 홈 쇼핑 들 , 동행 세일 동참",NNG,NNG,NNG,NNP,FCNNG FTNNP 전국 적 인 소비 붐 조성 기여 할 예정 LCNNG LTNNG,1.0,0.066667


In [10]:
train_df.head()

,n_id,date,title,content,ord,info_,clean_content,clean_title,mecab_,mecab_title,last_pos_content,first_pos_content,last_pos_title,first_pos_title,new,counts,ord_div
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0,[이데일리 MARKETPOINT] time 현재 코스닥 기관 money 순매도,[마감]코스닥 기관 money 순매도,[ 이데일리 MARKETPOINT ] time 현재 코스닥 기관 money 순매도,[ 마감 ] 코스닥 기관 money 순매도,NNG,SSO,NNG,SSO,FCSSO FTSSO [ 이데일리 MARKETPOINT ] time 현재 코스닥 기...,1.0,0.250000
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1,'실적기반' 저가에 매집해야 할 month 급등유망주 rank 전격공개,[마감]코스닥 기관 money 순매도,' 실적 기반 ' 저가 매집 해야 할 month 급등 유망주 rank 전격 공개,[ 마감 ] 코스닥 기관 money 순매도,NNG,SY,NNG,SSO,FCSY FTSSO ' 실적 기반 ' 저가 매집 해야 할 month 급등 유망주 r...,542.0,0.500000
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1,"하이스탁론, 선취수수료 없는 월 perc 최저금리 상품 출시",[마감]코스닥 기관 money 순매도,"하이스탁론 , 선취 수수료 없 는 월 perc 최저 금리 상품 출시",[ 마감 ] 코스닥 기관 money 순매도,NNG,NNP,NNG,SSO,"FCNNP FTSSO 하이스탁론 , 선취 수수료 없 는 월 perc 최저 금리 상품...",542.0,0.750000
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0,종합 경제정보 미디어 이데일리 무단전재 & 재배포 금지,[마감]코스닥 기관 money 순매도,종합 경제 정보 미디어 이데일리 무단 전재 & 재배포 금지,[ 마감 ] 코스닥 기관 money 순매도,NNG,NNG,NNG,SSO,FCNNG FTSSO 종합 경제 정보 미디어 이데일리 무단 전재 & 재배포 금지 L...,605.0,1.000000
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0,전국적인 소비 붐 조성에 기여할 예정,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국 적 인 소비 붐 조성 기여 할 예정,"롯데 · 공영 등 7 개 TV 홈 쇼핑 들 , 동행 세일 동참",NNG,NNG,NNG,NNP,FCNNG FTNNP 전국 적 인 소비 붐 조성 기여 할 예정 LCNNG LTNNG,1.0,0.066667


In [80]:
train_df.tail()

,n_id,date,title,content,ord,info_,clean_content,clean_title,mecab_,mecab_title,last_pos_content,first_pos_content,last_pos_title,first_pos_title,new,counts,ord_div
118740,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",미 FDA 임상3상 허가 임박. 묻고 따블로 갈 바이오 황제주.,48,1,미 FDA 임상3상 허가 임박 묻고 따블로 갈 바이오 황제주,"모나리자, 중원 지분 perc 양수 결정",미 FDA 임상3상 허가 임박 묻 고 따 블 갈 바이오 황제주,"모나리자 , 중원 지분 perc 양수 결정",NNP,NNG,NNG,NNG,FCNNG FTNNG 미 FDA 임상3상 허가 임박 묻 고 따 블 갈 바이오 황제주...,240.0,0.923077
118741,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",똑똑해진 소비자..한국도 이젠 소형차 시대,49,1,똑똑해진 소비자 Ellipsis 한국도 이젠 소형차 시대,"모나리자, 중원 지분 perc 양수 결정",똑똑 해진 소비자 Ellipsis 한국 이젠 소형차 시대,"모나리자 , 중원 지분 perc 양수 결정",NNG,XR,NNG,NNG,FCXR FTNNG 똑똑 해진 소비자 Ellipsis 한국 이젠 소형차 시대 LCN...,2420.0,0.942308
118742,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",똑똑해진 소비자..한국도 이젠 소형차 시대,50,1,똑똑해진 소비자 Ellipsis 한국도 이젠 소형차 시대,"모나리자, 중원 지분 perc 양수 결정",똑똑 해진 소비자 Ellipsis 한국 이젠 소형차 시대,"모나리자 , 중원 지분 perc 양수 결정",NNG,XR,NNG,NNG,FCXR FTNNG 똑똑 해진 소비자 Ellipsis 한국 이젠 소형차 시대 LCN...,2420.0,0.961538
118743,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,51,1,year 한국 TV 2대중 1대 인터넷 연결된다,"모나리자, 중원 지분 perc 양수 결정",year 한국 TV 2 대중 1 대 인터넷 연결 된다,"모나리자 , 중원 지분 perc 양수 결정",XSV+EC,SL,NNG,NNG,FCSL FTNNG year 한국 TV 2 대중 1 대 인터넷 연결 된다 LCXSV...,2420.0,0.980769
118744,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,52,1,year 한국 TV 2대중 1대 인터넷 연결된다,"모나리자, 중원 지분 perc 양수 결정",year 한국 TV 2 대중 1 대 인터넷 연결 된다,"모나리자 , 중원 지분 perc 양수 결정",XSV+EC,SL,NNG,NNG,FCSL FTNNG year 한국 TV 2 대중 1 대 인터넷 연결 된다 LCXSV...,2420.0,1.000000


## tf-idf
---


In [13]:
# 'n_id','date','title','content' 중복 제거
print('train data 개수: {}'.format(len(train_df)))

train_dup = train_df.drop_duplicates(['n_id','date','title','content'])
train_dup.reset_index(drop=True,inplace=True)

print('중복제거후 train data 개수: {}'.format(len(train_dup)))

train data 개수: 118745
중복제거후 train data 개수: 71583


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


def split_tokenizer(text):
    tokens_ko = text.split()
    return tokens_ko

tfidf_vect = TfidfVectorizer(ngram_range=(1,2), tokenizer=split_tokenizer,
                             min_df=3, max_df=0.9,analyzer = 'word')
tfidf_vect.fit(train_dup['new'])
tfidf_matrix_train = tfidf_vect.transform(train_df['new'])

C:\Users\67416\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [15]:
tfidf_matrix_train.shape

(118745, 84431)

In [16]:
# tf-idf feature 이름 확인
tfidf_vect.get_feature_names()[:5]

['"', '" "', '" (', '" 2', '" 4']

### ord_div, counts 변수를 sparse 형태로 tfidf_matrix_train와 결합

In [17]:
import scipy.sparse as sp

sparse_matrix = sp.csr_matrix(train_df[['ord_div','counts']].T)
tfidf_matrix_train = sp.hstack((sparse_matrix.T,tfidf_matrix_train))
tfidf_matrix_train.shape

(118745, 84433)

# LGBM
---

In [18]:
from sklearn.model_selection import train_test_split

# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수.
def get_train_test_dataset(df=None):
    
    # DataFrame의 맨 마지막 컬럼이 레이블, 나머지는 피처들
    X_features = tfidf_matrix_train
    y_target = df['info_']
    
    # train_test_split( )으로 학습과 테스트 데이터 분할. stratify=y_target으로 Stratified 기반 분할
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.3, random_state=25, stratify=y_target)
    
    # 학습과 테스트 데이터 세트 반환
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_train_test_dataset(train_df)


In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

# 모델의 성능을 측정하는 함수
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)    
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [20]:
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:, 1]
    get_clf_eval(tgt_test, pred, pred_proba)

## 하이퍼 파라미터 튜닝후 최종 모델 학습
---
- 하이퍼 파라미터 튜닝 과정을 담은 코드는 담지 않았습니다.

In [26]:
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from lightgbm import LGBMClassifier
tune_lgbm = LGBMClassifier(boosting_type= 'gbdt', colsample_bytree= 0.8, learning_rate= 0.05, 
                           n_estimators= 1000, scale_pos_weight= 1.1,random_state=25,n_jobs=-1)
#tune_lgbm.fit(tfidf_matrix_train,train_df.info_)
get_model_train_eval(tune_lgbm,X_train,X_test,y_train,y_test)

오차 행렬
[[21490    54]
 [  112 13968]]
정확도: 0.9953, 정밀도: 0.9961, 재현율: 0.9920,    F1: 0.9941, AUC:0.9998


## 학습된 모델과, tf-idf로 변환된 데이터를 pickle형태로 저장.
---

In [ ]:
import pickle

with open('tfidf.pickle', 'wb') as f:
    pickle.dump(tfidf_matrix_train, f, pickle.HIGHEST_PROTOCOL)

with open('LGBM.pickle', 'wb') as f:
    pickle.dump(tune_lgbm, f, pickle.HIGHEST_PROTOCOL)